In [1]:
#Step 1: Data Transformation
import pandas as pd
file_path='/Users/chloe/Desktop/patient_id_month_year - patient_id_month_year.csv'
df=pd.read_csv(file_path)

#check month_year format
print(type(df.loc[0, 'month_year']))

<class 'str'>


In [2]:
#convert month_year from string to date time format 
df['month_year']=pd.to_datetime(df['month_year'])

#sort values before calculate date gaps
df=df.sort_values(['patient_id', 'month_year'])

In [3]:
#Create an episode break indicator called 'break'
#for each patient_id, convert the month_year to a period, then calculate the month difference of month_year between the current row and previous row. 
#Transform the month difference calculation result to an integer 
#For the first row of each new patient_id, the difference will be null. Replace the null with 1 so it doesn't falsely identify as a break point. 
#If the difference doesn't equal to 1, label that row as a break point 
df['break']=df.groupby('patient_id')['month_year'].transform(lambda s:s.dt.to_period('M').astype(int).diff().fillna(1).ne(1))
print(df)

     patient_id month_year  break
0        ID0001 2023-01-01  False
1        ID0001 2023-02-01  False
2        ID0001 2023-03-01  False
3        ID0001 2023-06-01   True
4        ID0001 2023-08-01   True
...         ...        ...    ...
7304     ID1000 2023-02-01  False
7305     ID1000 2023-05-01   True
7306     ID1000 2023-07-01   True
7307     ID1000 2023-10-01   True
7308     ID1000 2023-11-01  False

[7309 rows x 3 columns]


In [4]:
#for each patient_id, assign each episode an ID 
df['episode_id']=df.groupby('patient_id')['break'].cumsum()
print(df)

     patient_id month_year  break  episode_id
0        ID0001 2023-01-01  False           0
1        ID0001 2023-02-01  False           0
2        ID0001 2023-03-01  False           0
3        ID0001 2023-06-01   True           1
4        ID0001 2023-08-01   True           2
...         ...        ...    ...         ...
7304     ID1000 2023-02-01  False           0
7305     ID1000 2023-05-01   True           1
7306     ID1000 2023-07-01   True           2
7307     ID1000 2023-10-01   True           3
7308     ID1000 2023-11-01  False           3

[7309 rows x 4 columns]


In [5]:
#for each patient_id and enrollment episode, calculate the first and last month 
#then reset the index, drop episode_id column as it's no longer needed 
span=(df.groupby(['patient_id', 'episode_id']).agg(enrollment_start_date=('month_year', 'min'),enrollment_end_date=('month_year', 'max')).
    reset_index().drop(columns='episode_id'))

In [6]:
#take the last day of each enrollment_end_date 
span['enrollment_end_date']=span['enrollment_end_date']+pd.offsets.MonthEnd(0)
print(span)

     patient_id enrollment_start_date enrollment_end_date
0        ID0001            2023-01-01          2023-03-31
1        ID0001            2023-06-01          2023-06-30
2        ID0001            2023-08-01          2023-08-31
3        ID0001            2023-11-01          2023-12-31
4        ID0002            2023-02-01          2023-04-30
...         ...                   ...                 ...
3100     ID0999            2023-12-01          2023-12-31
3101     ID1000            2023-02-01          2023-02-28
3102     ID1000            2023-05-01          2023-05-31
3103     ID1000            2023-07-01          2023-07-31
3104     ID1000            2023-10-01          2023-11-30

[3105 rows x 3 columns]


In [7]:
#save output to csv 
span.to_csv("/Users/chloe/Desktop/patient_enrollment_span.csv",index=False)
#3105 rows 

In [8]:
#Step 2: Data Aggregation
#load outpatient_visits_file
file_path2='/Users/chloe/Desktop/outpatient_visits_file - outpatient_visits_file.csv'
op=pd.read_csv(file_path2)
print(op)

      patient_id        date  outpatient_visit_count
0         ID0001    1/3/2023                       2
1         ID0001    1/8/2023                       2
2         ID0001    1/9/2023                       2
3         ID0001   1/15/2023                       2
4         ID0001   1/21/2023                       4
...          ...         ...                     ...
15323     ID0980  11/25/2023                       2
15324     ID0980  10/20/2023                       1
15325     ID0980    4/4/2023                       1
15326     ID0980   7/26/2023                       4
15327     ID0980   2/11/2023                       4

[15328 rows x 3 columns]


In [9]:
#check date column type 
print(type(op.loc[0, 'date']))
print(type(op.loc[0, 'outpatient_visit_count']))

<class 'str'>
<class 'numpy.int64'>


In [10]:
#check enorllment date column type 
span[['enrollment_start_date', 'enrollment_end_date']].dtypes

enrollment_start_date    datetime64[ns]
enrollment_end_date      datetime64[ns]
dtype: object

In [11]:
#convert date from string to datetime format 
op['date']=pd.to_datetime(op['date'])

In [12]:
#join span and op on patient_id and requires visit date between enrollment start date and end date 
filtered=span.merge(op, on='patient_id').query('enrollment_start_date <= date <= enrollment_end_date')

In [13]:
#aggregate the output: count unique days with op visit per patient per enrollement period; sum up visit count per patient per enrollment period 
result=(filtered.groupby(['patient_id', 'enrollment_start_date', 'enrollment_end_date'])
                .agg(ct_days_with_outpatient_visit=('date', 'nunique'),ct_outpatient_visits=('outpatient_visit_count', 'sum'))
                .reset_index()
                .sort_values(['patient_id','enrollment_start_date','enrollment_end_date'])
       )
print(result)

     patient_id enrollment_start_date enrollment_end_date  \
0        ID0001            2023-01-01          2023-03-31   
1        ID0001            2023-06-01          2023-06-30   
2        ID0001            2023-08-01          2023-08-31   
3        ID0001            2023-11-01          2023-12-31   
4        ID0002            2023-02-01          2023-04-30   
...         ...                   ...                 ...   
2025     ID0980            2023-02-01          2023-02-28   
2026     ID0980            2023-04-01          2023-04-30   
2027     ID0980            2023-07-01          2023-07-31   
2028     ID0980            2023-09-01          2023-09-30   
2029     ID0980            2023-11-01          2023-12-31   

      ct_days_with_outpatient_visit  ct_outpatient_visits  
0                                 8                    17  
1                                 2                     6  
2                                 2                     6  
3                          

In [14]:
#save result to csv 
result.to_csv("/Users/chloe/Desktop/result.csv",index=False)

In [15]:
#count unique values ct_days_with_outpatient_visit 
unique_counts=result['ct_days_with_outpatient_visit'].nunique()
print(unique_counts) 

32
